In [1]:
import subprocess
import os
from dotenv import load_dotenv
load_dotenv()

from transformers import RobertaForMaskedLM, RobertaTokenizerFast
import json
import pandas as pd
import glob


import numpy as np
from transformers import AutoTokenizer, AutoModel

# import sys
# sys.path.append("../UnMasked")

# from unmasked.mlm.scoring import mlm_score_model_on_paradigm
# from unmasked.holistic.scoring import holistic_score_model_on_paradigm
# from unmasked import configs
# from unmasked.utils import calc_accuracy_from_scores
# TODO batch size to 1

def eval(model_name = "loris3/stratified_10m_curriculum_random", EVAL_REPO_PATH = "../evaluation-pipeline-2024"):
    blimp_out_file = os.path.join("./results/blimp/", os.path.basename(model_name),"blimp_results.json")
    ewok_out_file = os.path.join("./results/ewok/", os.path.basename(model_name),"ewok_results.json")
    zorro_out_file = os.path.join("./results/zorro/", os.path.basename(model_name),"zorro.pkl")

    if (not os.path.exists(blimp_out_file)) or (not os.path.exists(ewok_out_file)):
        print("Running", os.path.basename(model_name))
        # local_path = os.path.join("./models", os.path.basename(model_name))
        # AutoModel.from_pretrained(model_name, cache_dir=local_path).save_pretrained(local_path)
        # AutoTokenizer.from_pretrained(model_name, cache_dir=local_path).save_pretrained(local_path)

        shellscript = subprocess.Popen([os.path.abspath("./eval"+("_llama" if "llama" in model_name else "") +".sh"), os.path.abspath(EVAL_REPO_PATH), os.path.abspath("./results"), model_name], stdin=subprocess.PIPE)

        shellscript.wait()

        if shellscript.returncode != 0:
            raise NotImplementedError
    blimp = None
    with open(blimp_out_file) as f:
        blimp = json.load(f)
    ewok = None
    with open(ewok_out_file) as f:
        ewok = json.load(f)


    # if (not os.path.exists(zorro_out_file) and "llama" not in model_name):
    #     os.makedirs(os.path.dirname(zorro_out_file))
    #     MODEL_REPO = model_name  # name of huggingface model hub repository
    #     LOWER_CASE = False  # should model be evaluated on lower-cased input?
    #     TEST_SUITE_NAME = ['zorro', 'blimp'][0]

    #     if TEST_SUITE_NAME == 'blimp':
    #         num_expected_scores = 2000
    #     elif TEST_SUITE_NAME == 'zorro':
    #         num_expected_scores = 4000
    #     else:
    #         raise AttributeError('Invalid "TEST_SUITE_NAME".')


    #     # each iteration in this "for" loop produces 1 row in the data-frame
    #     for scoring_method in ['mlm']:#, 'holistic']:

    #         if scoring_method == 'mlm':
    #             score_model_on_paradigm = mlm_score_model_on_paradigm
    #         elif scoring_method == 'holistic':
    #             score_model_on_paradigm = holistic_score_model_on_paradigm
    #         else:
    #             raise AttributeError('Invalid scoring_method.')

    #         # load from repo
    #         tokenizer = AutoTokenizer.from_pretrained(MODEL_REPO,
    #                                                 #   add_prefix_space=True,  # this must be True for BabyBERTa
    #                                                 )
    #         model = AutoModelForMaskedLM.from_pretrained(MODEL_REPO)

    #         model.eval()
    #         model.cuda(0)

    #         # for each paradigm in test suite
    #         accuracies = []
    #         for path_paradigm in (configs.Dirs.test_suites / TEST_SUITE_NAME).glob('*.txt'):

    #             # scoring
    #             print(f"Scoring {path_paradigm.name:<60} with {MODEL_REPO:<40} and method={scoring_method}")
    #             scores = score_model_on_paradigm(model, tokenizer, path_paradigm, lower_case=LOWER_CASE)

    #             assert len(scores) == num_expected_scores

    #             # compute accuracy
    #             accuracy = calc_accuracy_from_scores(scores, scoring_method)

    #             # collect
    #             accuracies.append(accuracy)
    #         df_zorro = pd.DataFrame.from_dict({"zorro":{
    #                                             "acc,none":  np.mean(accuracies),
    #                                             "acc_stderr,none": np.std(accuracies)}} ).T
    #         df_zorro.to_pickle(zorro_out_file)

            
    # if "llama" not in model_name:
    #     df = pd.concat([pd.DataFrame.from_dict(blimp["results"]).T, 
    #                         pd.DataFrame.from_dict(ewok["results"]).T, 
    #                         pd.read_pickle(zorro_out_file)/100])
    # else:
        df = pd.concat([pd.DataFrame.from_dict(blimp["results"]).T, 
                            pd.DataFrame.from_dict(ewok["results"]).T])
        
    df.index.name="metric"
    df["model"] = model_name
    df = df.drop("alias", axis=1)
    return df


RuntimeError: Failed to import transformers.models.roberta.modeling_roberta because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

TODO: from the findings:
`
Many masked language model submissions initially re
ported EWoK scores around 60–70%. This was likely due to a
 default behavior of the LM evaluation harness, which assigns
 a label of 0 when the probability of both sequences is the same.
 When changing this behavior to instead uniformly sample a
 label when the sequence probabilities are the same, most mod
els get closer to 50–60% accuracy. We confirmed these scores
 using a scoring script not based in the LM evaluation harness.
 This only affected EWoK: we were able to closely reproduce
 the participant-submitted scores for all other zero-shot tasks,
 with or without uniform sampling.
 `

In [ ]:
from huggingface_hub import list_models

model_names = [model.id for model in list_models(author="loris3") ]#if "stratified" in model.id]

In [ ]:
# from concurrent.futures import ThreadPoolExecutor


# with ThreadPoolExecutor(max_workers=1) as executor:
#     list(executor.map(eval, model_names))

In [ ]:
df = pd.concat([eval(model_name) for model_name in model_names])

Running stratified_10m_curriculum_llama_influential_examples_sandwich_llama


2024-12-20:07:18:36,823 INFO     [__main__.py:263] Verbosity set to INFO
2024-12-20:07:18:40,318 INFO     [__main__.py:349] Selected Tasks: ['blimp_filtered', 'blimp_supplement']
2024-12-20:07:18:40,320 INFO     [evaluator.py:133] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2024-12-20:07:18:40,320 INFO     [evaluator.py:181] Initializing hf model, with arguments: {'pretrained': 'loris3/stratified_10m_curriculum_llama_influential_examples_sandwich_llama', 'backend': 'causal'}


pretrained=loris3/stratified_10m_curriculum_llama_influential_examples_sandwich_llama,backend=causal


2024-12-20:07:18:41,816 INFO     [huggingface.py:196] Using device 'cuda:1'
2024-12-20:07:18:41,961 INFO     [huggingface.py:489] Overrode HF model backend type, and using type 'causal'
2024-12-20:07:18:46,294 WARNING  [task.py:788] [Task: blimp_complex_NP_island_filtered] metric acc is defined, but aggregation is not. using default aggregation=mean
2024-12-20:07:18:46,294 WARNING  [task.py:800] [Task: blimp_complex_NP_island_filtered] metric acc is defined, but higher_is_better is not. using default higher_is_better=True
2024-12-20:07:18:46,805 WARNING  [task.py:788] [Task: blimp_determiner_noun_agreement_with_adj_irregular_2_filtered] metric acc is defined, but aggregation is not. using default aggregation=mean
2024-12-20:07:18:46,805 WARNING  [task.py:800] [Task: blimp_determiner_noun_agreement_with_adj_irregular_2_filtered] metric acc is defined, but higher_is_better is not. using default higher_is_better=True
2024-12-20:07:18:47,291 WARNING  [task.py:788] [Task: blimp_wh_vs_that_n

hf (pretrained=loris3/stratified_10m_curriculum_llama_influential_examples_sandwich_llama,backend=causal), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: 1
|                                Tasks                                |Version|Filter|n-shot|Metric|Value |   |Stderr|
|---------------------------------------------------------------------|-------|------|-----:|------|-----:|---|-----:|
|blimp_supplement                                                     |N/A    |none  |     0|acc   |0.5291|±  |0.0060|
| - blimp_supplement_hypernym                                         |      1|none  |     0|acc   |0.4988|±  |0.0172|
| - blimp_supplement_qa_congruence_easy                               |      1|none  |     0|acc   |0.4531|±  |0.0627|
| - blimp_supplement_qa_congruence_tricky                             |      1|none  |     0|acc   |0.3939|±  |0.0382|
| - blimp_supplement_subject_aux_inversion                            |      1|none  |     0|acc   |0.7960|±  |0.

2024-12-20:07:29:49,103 INFO     [__main__.py:263] Verbosity set to INFO
2024-12-20:07:29:52,333 INFO     [__main__.py:349] Selected Tasks: ['ewok_filtered']
2024-12-20:07:29:52,335 INFO     [evaluator.py:133] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2024-12-20:07:29:52,335 INFO     [evaluator.py:181] Initializing hf model, with arguments: {'pretrained': 'loris3/stratified_10m_curriculum_llama_influential_examples_sandwich_llama', 'backend': 'causal'}


pretrained=loris3/stratified_10m_curriculum_llama_influential_examples_sandwich_llama,backend=causal


2024-12-20:07:29:54,279 INFO     [huggingface.py:196] Using device 'cuda:1'
2024-12-20:07:29:54,735 INFO     [huggingface.py:489] Overrode HF model backend type, and using type 'causal'
2024-12-20:07:29:56,510 WARNING  [task.py:788] [Task: ewok_social-relations_filtered] metric acc is defined, but aggregation is not. using default aggregation=mean
2024-12-20:07:29:56,510 WARNING  [task.py:800] [Task: ewok_social-relations_filtered] metric acc is defined, but higher_is_better is not. using default higher_is_better=True
2024-12-20:07:29:57,018 WARNING  [task.py:788] [Task: ewok_spatial-relations_filtered] metric acc is defined, but aggregation is not. using default aggregation=mean
2024-12-20:07:29:57,018 WARNING  [task.py:800] [Task: ewok_spatial-relations_filtered] metric acc is defined, but higher_is_better is not. using default higher_is_better=True
2024-12-20:07:29:57,420 WARNING  [task.py:788] [Task: ewok_physical-dynamics_filtered] metric acc is defined, but aggregation is not. us

hf (pretrained=loris3/stratified_10m_curriculum_llama_influential_examples_sandwich_llama,backend=causal), gen_kwargs: (None), limit: None, num_fewshot: None, batch_size: 1
|                 Tasks                  |Version|Filter|n-shot|Metric|Value |   |Stderr|
|----------------------------------------|-------|------|-----:|------|-----:|---|-----:|
|ewok_filtered                           |N/A    |none  |     0|acc   |0.4959|±  |0.0057|
| - ewok_agent-properties_filtered       |      1|none  |     0|acc   |0.4955|±  |0.0106|
| - ewok_material-dynamics_filtered      |      1|none  |     0|acc   |0.4987|±  |0.0180|
| - ewok_material-properties_filtered    |      1|none  |     0|acc   |0.4529|±  |0.0383|
| - ewok_physical-dynamics_filtered      |      1|none  |     0|acc   |0.4750|±  |0.0458|
| - ewok_physical-interactions_filtered  |      1|none  |     0|acc   |0.4964|±  |0.0212|
| - ewok_physical-relations_filtered     |      1|none  |     0|acc   |0.4976|±  |0.0175|
| - ewok_quantita

In [ ]:
from scipy.stats import ttest_ind_from_stats, ttest_rel

In [ ]:
def do_ttests(benchmark_name, files, model_names):
    dfs = [pd.concat([pd.read_json(os.path.join("./results/",benchmark_name.split("_")[0], os.path.basename(model_name),file)) for file in sorted(files)]) for model_name in model_names]
    data = sorted([(df["acc"].mean(),df["acc"]) for df in dfs], key= lambda l : l[0], reverse=True)
    return {benchmark_name:[ttest_rel(a,b, alternative="greater").pvalue for (mean_a,a),(mean_b,b) in zip(data, data[1:])]}

In [ ]:
filenames_supplement  = [os.path.basename(file) for file in sorted(glob.glob(os.path.join(os.path.join("./results/blimp", os.path.basename(model_names[0])), "blimp_supplement_*.jsonl")))]
filenames_filtered  = [os.path.basename(file)  for file in sorted(glob.glob(os.path.join(os.path.join("./results/blimp", os.path.basename(model_names[0])), "*.jsonl"))) if "supplement" not in file]
filenames_ewok  = [os.path.basename(file) for file in sorted(glob.glob(os.path.join(os.path.join("./results/ewok", os.path.basename(model_names[0])), "*.jsonl")))]


In [ ]:
df

,"acc,none","acc_stderr,none",model
metric,,,
blimp_supplement,0.593624,0.006536,loris3/stratified_10m_curriculum_random
blimp_supplement_hypernym,0.527316,0.017216,loris3/stratified_10m_curriculum_random
blimp_supplement_qa_congruence_easy,0.65625,0.059839,loris3/stratified_10m_curriculum_random
blimp_supplement_qa_congruence_tricky,0.527273,0.038985,loris3/stratified_10m_curriculum_random
blimp_supplement_subject_aux_inversion,0.689423,0.007442,loris3/stratified_10m_curriculum_random
...,...,...,...
ewok_quantitative-properties_filtered,0.5,0.028262,loris3/stratified_10m_curriculum_llama_influen...
ewok_social-interactions_filtered,0.510204,0.029204,loris3/stratified_10m_curriculum_llama_influen...
ewok_social-properties_filtered,0.521341,0.027625,loris3/stratified_10m_curriculum_llama_influen...


In [ ]:
p_vals = \
do_ttests("blimp_supplement",filenames_supplement, model_names) | \
do_ttests("blimp_filtered",filenames_filtered, model_names) | \
do_ttests("ewok_filtered",filenames_ewok, model_names)
p_vals

{'blimp_supplement': [0.2797576240649574,
  0.035119124755424294,
  0.4747132271757554,
  0.3004397308194905,
  0.0057655540012815625,
  0.46223392739813046,
  0.3243917504107078,
  0.36507420328216866,
  0.3902615215286992,
  0.4218573245067069,
  0.35178914904296577,
  0.000781421031326121,
  0.294396922570642,
  3.1146923250438456e-09,
  0.46626128823151514,
  7.877671089351814e-08,
  0.024855853864752434,
  0.2104404968814526,
  3.9931133176897857e-29],
 'blimp_filtered': [0.12334775558852588,
  0.23879694656421252,
  0.36117201867477333,
  0.21058537816949968,
  0.466450505365944,
  0.1450987236727158,
  0.0009623957876433912,
  0.2688501520239526,
  0.47203428733071867,
  0.204786581489451,
  0.052154403673944964,
  6.352886483779778e-05,
  0.2092155824387692,
  1.662476462337466e-10,
  0.008969134290406324,
  4.413455238272855e-12,
  2.0279135119351411e-41,
  0.20860653640568583,
  3.878682098848115e-45],
 'ewok_filtered': [0.040533017129168084,
  0.19388256207258153,
  0.206402

In [ ]:

df_results = df[df.index.isin(["blimp_supplement", "blimp_filtered", "ewok_filtered"])]#.set_index("model", append=True).groupby("model").apply(lambda a: a[:])
df_results = df_results.reset_index().set_index("model").pivot(columns="metric")
df_results.columns = df_results.columns.swaplevel(0)
df_results.columns = pd.MultiIndex.from_tuples([(c, b.replace(",none", "")) for c,b in df_results.columns])
df_results = pd.concat(
    {group: df_results.xs(group, axis=1, level=0) for group in df_results.columns.get_level_values(0).unique()},
    axis=1,
)
df_results

blimp_filtered             \
                                                              acc acc_stderr   
model                                                                          
loris3/babylm_2024_10m_curriculum_curriculum             0.574666   0.001891   
loris3/babylm_2024_10m_curriculum_random                 0.598839   0.001814   
loris3/stratified_10m_curriculum_curriculum              0.596632    0.00183   
loris3/stratified_10m_curriculum_dirac                   0.602535    0.00178   
loris3/stratified_10m_curriculum_dirac_positive...       0.588921   0.001839   
loris3/stratified_10m_curriculum_influential_ex...       0.522353   0.001864   
loris3/stratified_10m_curriculum_influential_ex...       0.613203   0.001766   
loris3/stratified_10m_curriculum_influential_ex...       0.612566   0.001771   
loris3/stratified_10m_curriculum_influential_ex...       0.600958   0.001793   
loris3/stratified_10m_curriculum_llama_influent...       0.616812   0.001832   
loris3/stratified_10m_curriculum_llama_random            0.480833   0.001961   
loris3/stratified_10m_curriculum_lognorm                 0.611121   0.001774   
loris3/stratified_10m_curriculum_lognorm_top_50...        0.56007   0.001871   
loris3/stratified_10m_curriculum_lognorm_top_50...       0.570286   0.001852   
loris3/stratified_10m_curriculum_lognorm_top_50...        0.60137   0.001812   
loris3/stratified_10m_curriculum_random                  0.611433   0.001774   
loris3/stratified_10m_curriculum_top_90_full_ep...       0.587918   0.001854   
loris3/stratified_equitoken_10m_curriculum_dirac         0.613969   0.001804   
loris3/stratified_equitoken_10m_curriculum_lognorm       0.608502   0.001806   
loris3/stratified_equitoken_10m_curriculum_random        0.524308   0.001907   

                                                   blimp_supplement  \
                                                                acc   
model                                                                 
loris3/babylm_2024_10m_curriculum_curriculum                0.54152   
loris3/babylm_2024_10m_curriculum_random                   0.593568   
loris3/stratified_10m_curriculum_curriculum                0.559653   
loris3/stratified_10m_curriculum_dirac                     0.628778   
loris3/stratified_10m_curriculum_dirac_positive...         0.620367   
loris3/stratified_10m_curriculum_influential_ex...         0.518928   
loris3/stratified_10m_curriculum_influential_ex...           0.6174   
loris3/stratified_10m_curriculum_influential_ex...         0.597881   
loris3/stratified_10m_curriculum_influential_ex...         0.580272   
loris3/stratified_10m_curriculum_llama_influent...         0.529083   
loris3/stratified_10m_curriculum_llama_random              0.459742   
loris3/stratified_10m_curriculum_lognorm                   0.593484   
loris3/stratified_10m_curriculum_lognorm_top_50...         0.604583   
loris3/stratified_10m_curriculum_lognorm_top_50...         0.576253   
loris3/stratified_10m_curriculum_lognorm_top_50...         0.602646   
loris3/stratified_10m_curriculum_random                    0.593624   
loris3/stratified_10m_curriculum_top_90_full_ep...         0.617415   
loris3/stratified_equitoken_10m_curriculum_dirac           0.622971   
loris3/stratified_equitoken_10m_curriculum_lognorm         0.634451   
loris3/stratified_equitoken_10m_curriculum_random          0.568102   

                                                              ewok_filtered  \
                                                   acc_stderr           acc   
model                                                                         
loris3/babylm_2024_10m_curriculum_curriculum         0.006902      0.679142   
loris3/babylm_2024_10m_curriculum_random             0.006392      0.667404   
loris3/stratified_10m_curriculum_curriculum          0.006679      0.635073   
loris3/stratified_10m_curriculum_dirac               0.006218      0.666974   
loris3/stratified_10m_c

In [ ]:
pd.DataFrame(df_results[df_results.columns[0::2]].mean(axis=1).sort_values(ascending=False))

,0
model,
loris3/stratified_10m_curriculum_influential_examples_first_half,0.63943
loris3/stratified_10m_curriculum_dirac,0.632762
loris3/stratified_equitoken_10m_curriculum_dirac,0.630674
loris3/stratified_10m_curriculum_influential_examples_first_third,0.629644
loris3/stratified_10m_curriculum_lognorm,0.626325
loris3/stratified_equitoken_10m_curriculum_lognorm,0.623917
loris3/stratified_10m_curriculum_top_90_full_epoch_shuffled,0.621956
loris3/stratified_10m_curriculum_lognorm_top_50_pct_twice_shuffled,0.621644
loris3/stratified_10m_curriculum_dirac_positive_only,0.621101


In [ ]:
for col in df_results.columns[0:5][0::2]:
    metric = col[0]
    df =df_results[[col[0]]].sort_values(by=df_results[[col]].columns[0], ascending=False)
    df[(metric,"p")] = [pd.NA] + p_vals[metric]
    display(df)

blimp_filtered             \
                                                              acc acc_stderr   
model                                                                          
loris3/stratified_10m_curriculum_llama_influent...       0.616812   0.001832   
loris3/stratified_equitoken_10m_curriculum_dirac         0.613969   0.001804   
loris3/stratified_10m_curriculum_influential_ex...       0.613203   0.001766   
loris3/stratified_10m_curriculum_influential_ex...       0.612566   0.001771   
loris3/stratified_10m_curriculum_random                  0.611433   0.001774   
loris3/stratified_10m_curriculum_lognorm                 0.611121   0.001774   
loris3/stratified_equitoken_10m_curriculum_lognorm       0.608502   0.001806   
loris3/stratified_10m_curriculum_dirac                   0.602535    0.00178   
loris3/stratified_10m_curriculum_lognorm_top_50...        0.60137   0.001812   
loris3/stratified_10m_curriculum_influential_ex...       0.600958   0.001793   
loris3/babylm_2024_10m_curriculum_random                 0.598839   0.001814   
loris3/stratified_10m_curriculum_curriculum              0.596632    0.00183   
loris3/stratified_10m_curriculum_dirac_positive...       0.588921   0.001839   
loris3/stratified_10m_curriculum_top_90_full_ep...       0.587918   0.001854   
loris3/babylm_2024_10m_curriculum_curriculum             0.574666   0.001891   
loris3/stratified_10m_curriculum_lognorm_top_50...       0.570286   0.001852   
loris3/stratified_10m_curriculum_lognorm_top_50...        0.56007   0.001871   
loris3/stratified_equitoken_10m_curriculum_random        0.524308   0.001907   
loris3/stratified_10m_curriculum_influential_ex...       0.522353   0.001864   
loris3/stratified_10m_curriculum_llama_random            0.480833   0.001961   

                                                              
                                                           p  
model                                                         
loris3/stratified_10m_curriculum_llama_influent...      <NA>  
loris3/stratified_equitoken_10m_curriculum_dirac    0.123348  
loris3/stratified_10m_curriculum_influential_ex...  0.238797  
loris3/stratified_10m_curriculum_influential_ex...  0.361172  
loris3/stratified_10m_curriculum_random             0.210585  
loris3/stratified_10m_curriculum_lognorm            0.466451  
loris3/stratified_equitoken_10m_curriculum_lognorm  0.145099  
loris3/stratified_10m_curriculum_dirac              0.000962  
loris3/stratified_10m_curriculum_lognorm_top_50...   0.26885  
loris3/stratified_10m_curriculum_influential_ex...  0.472034  
loris3/babylm_2024_10m_curriculum_random            0.204787  
loris3/stratified_10m_curriculum_curriculum         0.052154  
loris3/stratified_10m_curriculum_dirac_positive...  0.000064  
loris3/stratified_10m_curriculum_top_90_full_ep...  0.209216  
loris3/babylm_2024_10m_curriculum_curriculum             0.0  
loris3/stratified_10m_curriculum_lognorm_top_50...  0.008969  
loris3/stratified_10m_curriculum_lognorm_top_50...       0.0  
loris3/stratified_equitoken_10m_curriculum_random        0.0  
loris3/stratified_10m_curriculum_influential_ex...  0.208607  
loris3/stratified_10m_curriculum_llama_random            0.0

blimp_supplement  \
                                                                acc   
model                                                                 
loris3/stratified_equitoken_10m_curriculum_lognorm         0.634451   
loris3/stratified_10m_curriculum_dirac                     0.628778   
loris3/stratified_equitoken_10m_curriculum_dirac           0.622971   
loris3/stratified_10m_curriculum_dirac_positive...         0.620367   
loris3/stratified_10m_curriculum_top_90_full_ep...         0.617415   
loris3/stratified_10m_curriculum_influential_ex...           0.6174   
loris3/stratified_10m_curriculum_lognorm_top_50...         0.604583   
loris3/stratified_10m_curriculum_lognorm_top_50...         0.602646   
loris3/stratified_10m_curriculum_influential_ex...         0.597881   
loris3/stratified_10m_curriculum_random                    0.593624   
loris3/babylm_2024_10m_curriculum_random                   0.593568   
loris3/stratified_10m_curriculum_lognorm                   0.593484   
loris3/stratified_10m_curriculum_influential_ex...         0.580272   
loris3/stratified_10m_curriculum_lognorm_top_50...         0.576253   
loris3/stratified_equitoken_10m_curriculum_random          0.568102   
loris3/stratified_10m_curriculum_curriculum                0.559653   
loris3/babylm_2024_10m_curriculum_curriculum                0.54152   
loris3/stratified_10m_curriculum_llama_influent...         0.529083   
loris3/stratified_10m_curriculum_influential_ex...         0.518928   
loris3/stratified_10m_curriculum_llama_random              0.459742   

                                                                         
                                                   acc_stderr         p  
model                                                                    
loris3/stratified_equitoken_10m_curriculum_lognorm   0.006356      <NA>  
loris3/stratified_10m_curriculum_dirac               0.006218  0.279758  
loris3/stratified_equitoken_10m_curriculum_dirac     0.006429  0.035119  
loris3/stratified_10m_curriculum_dirac_positive...   0.006206  0.474713  
loris3/stratified_10m_curriculum_top_90_full_ep...   0.006388   0.30044  
loris3/stratified_10m_curriculum_influential_ex...   0.006362  0.005766  
loris3/stratified_10m_curriculum_lognorm_top_50...   0.006569  0.462234  
loris3/stratified_10m_curriculum_lognorm_top_50...   0.006229  0.324392  
loris3/stratified_10m_curriculum_influential_ex...   0.006111  0.365074  
loris3/stratified_10m_curriculum_random              0.006536  0.390262  
loris3/babylm_2024_10m_curriculum_random             0.006392  0.421857  
loris3/stratified_10m_curriculum_lognorm             0.006342  0.351789  
loris3/stratified_10m_curriculum_influential_ex...   0.006349  0.000781  
loris3/stratified_10m_curriculum_lognorm_top_50...   0.006687  0.294397  
loris3/stratified_equitoken_10m_curriculum_random    0.006893       0.0  
loris3/stratified_10m_curriculum_curriculum          0.006679  0.466261  
loris3/babylm_2024_10m_curriculum_curriculum         0.006902       0.0  
loris3/stratified_10m_curriculum_llama_influent...   0.005952  0.024856  
loris3/stratified_10m_curriculum_influential_ex...   0.006791   0.21044  
loris3/stratified_10m_curriculum_llama_random        0.006823       0.0

ewok_filtered             \
                                                             acc acc_stderr   
model                                                                         
loris3/stratified_10m_curriculum_influential_ex...      0.687686   0.004736   
loris3/babylm_2024_10m_curriculum_curriculum            0.679142    0.00484   
loris3/stratified_10m_curriculum_influential_ex...      0.678484   0.004773   
loris3/stratified_10m_curriculum_lognorm                 0.67437    0.00484   
loris3/babylm_2024_10m_curriculum_random                0.667404   0.004859   
loris3/stratified_10m_curriculum_dirac                  0.666974   0.004916   
loris3/stratified_10m_curriculum_lognorm_top_50...      0.664983   0.004902   
loris3/stratified_10m_curriculum_lognorm_top_50...      0.660918   0.004912   
loris3/stratified_10m_curriculum_lognorm_top_50...      0.660767   0.004945   
loris3/stratified_10m_curriculum_top_90_full_ep...      0.660534   0.004878   
loris3/stratified_10m_curriculum_influential_ex...      0.658878   0.004932   
loris3/stratified_equitoken_10m_curriculum_random          0.656   0.005008   
loris3/stratified_equitoken_10m_curriculum_dirac        0.655081   0.004858   
loris3/stratified_10m_curriculum_dirac_positive...      0.654016   0.004857   
loris3/stratified_10m_curriculum_random                 0.646186   0.004838   
loris3/stratified_10m_curriculum_influential_ex...      0.636234   0.004931   
loris3/stratified_10m_curriculum_curriculum             0.635073   0.004923   
loris3/stratified_equitoken_10m_curriculum_lognorm      0.628797   0.004831   
loris3/stratified_10m_curriculum_llama_influent...      0.495912   0.005732   
loris3/stratified_10m_curriculum_llama_random           0.488865   0.005731   

                                                              
                                                           p  
model                                                         
loris3/stratified_10m_curriculum_influential_ex...      <NA>  
loris3/babylm_2024_10m_curriculum_curriculum        0.040533  
loris3/stratified_10m_curriculum_influential_ex...  0.193883  
loris3/stratified_10m_curriculum_lognorm            0.206403  
loris3/babylm_2024_10m_curriculum_random            0.289773  
loris3/stratified_10m_curriculum_dirac              0.445178  
loris3/stratified_10m_curriculum_lognorm_top_50...  0.458571  
loris3/stratified_10m_curriculum_lognorm_top_50...  0.401243  
loris3/stratified_10m_curriculum_lognorm_top_50...  0.443524  
loris3/stratified_10m_curriculum_top_90_full_ep...  0.482095  
loris3/stratified_10m_curriculum_influential_ex...   0.41244  
loris3/stratified_equitoken_10m_curriculum_random   0.375677  
loris3/stratified_equitoken_10m_curriculum_dirac    0.422995  
loris3/stratified_10m_curriculum_dirac_positive...  0.403352  
loris3/stratified_10m_curriculum_random             0.401921  
loris3/stratified_10m_curriculum_influential_ex...  0.204616  
loris3/stratified_10m_curriculum_curriculum              0.5  
loris3/stratified_equitoken_10m_curriculum_lognorm  0.474275  
loris3/stratified_10m_curriculum_llama_influent...       0.0  
loris3/stratified_10m_curriculum_llama_random       0.389596